In [14]:
%matplotlib inline
import sys
sys.path.insert(0, "../..")

<!--<badge>--><a href="https://colab.research.google.com/github/softmatterlab/DeepTrack-2.0/blob/bm/improve-mie/examples/module-examples/properties_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

# deeptrack.properties

This example introduces the module deeptrack.properties.

## What is a property?

Each feature (instance of the class `Feature`, see [features_example](features_example.ipynb)) can have several properties (instances of the class `Property`).
A propety has a value accessible through the field `current_value`, whose data type is not restricted. 
This value is updated through a sampling rule (method `.update()`), which is passed to the class constructor on initialization. 

## What is a sampling rule?

The sampling rule determines how the value of a property is updated upon calling `.update()`.
A sampling rule is defined when an instance of the class Property is created and can be of any type. 
When calling `.update()`, the value of the property is updated according to the first of the following that applies:
    
1.  If the sampling rule has a method `.sample()`, call `.sample()` and return the output.
2.  If the sampling rule is a ``dict``, sample each value and combine the result into a new ``dict`` using the original keys. 
3.  If the sampling rule is a ``list``, sample each element of the list and combine the result into a new ``list``.
4.  If the sampling rule is an iterable, return the next value. If the iterable is exhausted, the value of the property is not changed.
5.  If the sampling rule is callable, call it and return the result.
6.  If none of the above apply, return the sampling rule itself.

In [15]:
import numpy as np
from deeptrack.properties import Property, SequentialProperty, PropertyDict

## 1.1 Property with a constant value

The simplest example of a property is one that does not change during an update call.
This is commonly either a number or a tuple, but can be any data type that will be evaluated by case 6.
If you want to have a constant property with a value that would be evaluated by cases 1-5 (e.g., a list or a function), you can  wrap it as the output of a lambda function.

In [17]:
# NUMBER

P = Property(1)
print("The current value of the property is", P())

P.update() # Numbers are not changed after an update() call
print("The current value of the property is", P())

The current value of the property is 1
The current value of the property is 1


In [18]:
# TUPLE

P = Property((1, [2, 3], None))
print("The current value of the property is", P())

P.update() # Tuples are not changed after an update() call
print("The current value of the property is", P())

The current value of the property is (1, [2, 3], None)
The current value of the property is (1, [2, 3], None)


In [19]:
# WRAPPED LIST

P = Property(lambda: [np.random.rand, 1, {}])
print("The current value of the property is", P())

P.update() # Objects wrapped in functions are not changed after an update() call
print("The current value of the property is", P())

The current value of the property is [<built-in method rand of numpy.random.mtrand.RandomState object at 0x0000022AA23D2740>, 1, {}]
The current value of the property is [<built-in method rand of numpy.random.mtrand.RandomState object at 0x0000022AA23D2740>, 1, {}]


## 1.2 Property with a discrete random value 

Discrete randomness can be achieved by a function (case 5).

In [20]:
# FUNCTION

P = Property(lambda: np.random.randint(0, 10))
for _ in range(5): 
    P.update()
    print("The current value of the property is", P())

The current value of the property is 9
The current value of the property is 5
The current value of the property is 0
The current value of the property is 3
The current value of the property is 0


In [26]:
# BINARY CHOICE

P = Property(lambda: 1 if np.random.rand() > 0.75 else 0)
for _ in range(5): 
    P.update()
    print("The current value of the property is", P())

The current value of the property is 0
The current value of the property is 0
The current value of the property is 0
The current value of the property is 0
The current value of the property is 0


## 1.3 Property with a continuous random value 

Continuous randomness is typically achieved by passing a function that returns a coninuous random value. This function should take no input, as noted in case 5. To use a function that needs arguments, wrap it in a function that calls it with the correct arguments.

In [28]:
# FUNCTION WITH NO INPUT

P = Property(np.random.rand)
for _ in range(5):
    P.update()
    print("The current value of the property is", P())

The current value of the property is 0.5037658050216487
The current value of the property is 0.5479732300575825
The current value of the property is 0.4763315193624603
The current value of the property is 0.22705584756076214
The current value of the property is 0.6441702905607429


In [29]:
# WRAPPED FUNCTION

P = Property(lambda: np.random.normal(1, 5))
for _ in range(5):
    P.update()
    print("The current value of the property is", P())

The current value of the property is -1.8974331729875393
The current value of the property is 8.431663783659689
The current value of the property is 7.968554968599136
The current value of the property is 3.0520276250778795
The current value of the property is 3.2728790741261036


## 1.4 Property with a deterministically changing value

Deterministically changing properties can be achieved using either an iterator (case 4) or a function (case 5). For the output of a function to change deterministically between calls, it should reference some variable outside its definition. Once an iterator has been exhausted, it will always return its last value.

In [30]:
# ITERATOR

P = Property(iter([1, 2, 3, 4, 5]))
for _ in range(10):
    P.update()
    print("The current value of the property is", P())

The current value of the property is 1
The current value of the property is 2
The current value of the property is 3
The current value of the property is 4
The current value of the property is 5
The current value of the property is 5
The current value of the property is 5
The current value of the property is 5
The current value of the property is 5
The current value of the property is 5


In [31]:
# FUNCTION

fibbonacci = [1, 1]
def fibbonacci_sequence():
    fibbonacci.append(fibbonacci[-2] + fibbonacci[-1])
    return fibbonacci

P = Property(fibbonacci_sequence)
for _ in range(10):
    P.update()
    print("The current value of the property is", P())

The current value of the property is [1, 1, 2]
The current value of the property is [1, 1, 2, 3]
The current value of the property is [1, 1, 2, 3, 5]
The current value of the property is [1, 1, 2, 3, 5, 8]
The current value of the property is [1, 1, 2, 3, 5, 8, 13]
The current value of the property is [1, 1, 2, 3, 5, 8, 13, 21]
The current value of the property is [1, 1, 2, 3, 5, 8, 13, 21, 34]
The current value of the property is [1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
The current value of the property is [1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]
The current value of the property is [1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144]


## 1.5 Property with dependent value

The value of a property can be dependent on the value on some other property. It does this by accepting some keyword argument corresponding to the name of the independent property. Instances of `Feature` will handle this automatically.

In [35]:
random_number = Property(lambda: np.random.rand())

def get_dependent_number():
    return random_number() + 1

dependent_number = Property(get_dependent_number)

# Link the properties. This is handled automatically by PropertyDict
dependent_number.add_dependency(random_number)
random_number.add_child(dependent_number)

for _ in range(5):
    dependent_number.update()
    
    print("The current value of the independent property is", random_number() )
    print("The current value of the dependent property is", dependent_number(), "\n")
    


The current value of the independent property is 0.011554854471887421
The current value of the dependent property is 1.0115548544718873 

The current value of the independent property is 0.19367374949954808
The current value of the dependent property is 1.1936737494995482 

The current value of the independent property is 0.9344759767385629
The current value of the dependent property is 1.9344759767385629 

The current value of the independent property is 0.9089543321605803
The current value of the dependent property is 1.9089543321605804 

The current value of the independent property is 0.6999097121055312
The current value of the dependent property is 1.6999097121055313 



## 3. What is a PropertyDict?

Another class contained in the module deeptrack.properties is `PropertyDict`. This is a dictionary of properties (keys: name of properties; values: properties) complemented by utility methods to manage collections of properties. These include:

* `.current_value_dict()`, which creates and returns a dictionary with the current value of all properties in the PropertyDict (keys: name of properties; values: current values of the properties).
* `.update()`, which calls the method `.update()` on all properties in the PropertyDict.
* `.sample()`, which calls the method `.sample()` on all properties in the PropertyDict, and creates and returns a dictionary from the output (keys: name of properties; values: sample outputs of the properties).


In [42]:
property_dict = PropertyDict(
    foo="foo",
    barorbaz=lambda:np.random.choice(["bar", "baz"]),
    foobarorbaz=lambda foo, barorbaz: foo + barorbaz
)

for _ in range(5):
    property_dict.update()
    print("The current values of the properties in property_dict are", property_dict())

The current values of the properties in property_dict are {'foo': 'foo', 'barorbaz': 'baz', 'foobarorbaz': 'foobaz'}
The current values of the properties in property_dict are {'foo': 'foo', 'barorbaz': 'baz', 'foobarorbaz': 'foobaz'}
The current values of the properties in property_dict are {'foo': 'foo', 'barorbaz': 'bar', 'foobarorbaz': 'foobar'}
The current values of the properties in property_dict are {'foo': 'foo', 'barorbaz': 'bar', 'foobarorbaz': 'foobar'}
The current values of the properties in property_dict are {'foo': 'foo', 'barorbaz': 'baz', 'foobarorbaz': 'foobaz'}
